# Joe's code place

![Hackerman](https://i.kym-cdn.com/entries/icons/mobile/000/021/807/ig9OoyenpxqdCQyABmOQBZDI0duHk2QZZmWg2Hxd4ro.jpg)

# Code

In [1]:
# Imports
import os
import pandas as pd
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
""" Ticker Library init
portfolio = {'Sector':[],'Company Name':[],'Ticker':[]}
hello = [
    ['Technology','Netflix','NFLX'],
    ['Technology','Meta','FB'],
    ['Technology','Uber','UBER'],
    ['Technology','Microchip Technology','MCHP'],
    ['Technology','AirBnB','ABNB'],
    ['Energy','Diamondback Energy Inc.','FANG'],
    ['Energy','Marathon Oil Corp.','MRO'],
    ['Energy','Devon Energy Corp.','DVN'],
    ['Energy','SunPower Corp.','SPWR'],
    ['Energy','Renewable Energy Group Inc.','REGI'],
    ['Finance','McKinsey & Company','MTRX'],
    ['Finance','BlackRock','BLK'],
    ['Finance','PayPal','PYPL'],
    ['Finance','Mercadolibre Inc','MELI'],
    ['Finance','SoFi','SOFI']
]
for row in hello:
    portfolio['Sector'].append(row[0])
    portfolio['Company Name'].append(row[1])
    portfolio['Ticker'].append(row[2])
portfolio = pd.DataFrame(portfolio).set_index('Sector')
portfolio.to_csv('./Data/Cleaned_Data/Ticker_library.csv')
"""

" Ticker Library init\nportfolio = {'Sector':[],'Company Name':[],'Ticker':[]}\nhello = [\n    ['Technology','Netflix','NFLX'],\n    ['Technology','Meta','FB'],\n    ['Technology','Uber','UBER'],\n    ['Technology','Microchip Technology','MCHP'],\n    ['Technology','AirBnB','ABNB'],\n    ['Energy','Diamondback Energy Inc.','FANG'],\n    ['Energy','Marathon Oil Corp.','MRO'],\n    ['Energy','Devon Energy Corp.','DVN'],\n    ['Energy','SunPower Corp.','SPWR'],\n    ['Energy','Renewable Energy Group Inc.','REGI'],\n    ['Finance','McKinsey & Company','MTRX'],\n    ['Finance','BlackRock','BLK'],\n    ['Finance','PayPal','PYPL'],\n    ['Finance','Mercadolibre Inc','MELI'],\n    ['Finance','SoFi','SOFI']\n]\nfor row in hello:\n    portfolio['Sector'].append(row[0])\n    portfolio['Company Name'].append(row[1])\n    portfolio['Ticker'].append(row[2])\nportfolio = pd.DataFrame(portfolio).set_index('Sector')\nportfolio.to_csv('./Data/Cleaned_Data/Ticker_library.csv')\n"

## Twitter

In [3]:
import tweepy

In [4]:
def init_twitter_API():
    return tweepy.API(
        tweepy.OAuth1UserHandler(
            # Generated when you setup your Twitter Dev account
            os.getenv("twitter_api_consumer_key"),
            os.getenv("twitter_api_consumer_secret_key"),
            # You have to generate the below after you setup your Twitter Dev Account
            os.getenv("twitter_api_access_key"),
            os.getenv("twitter_api_access_secret_key")
        )
    )

# tw = init_twitter_API()

## Python Edgar

In [5]:
tickers = pd.read_csv('./Data/Cleaned_Data/Ticker_library.csv')['Ticker'].to_list()

In [6]:
from sec_edgar_downloader import Downloader

In [7]:
def create_SEC(path,tickers,filing_type,after_date,before_date=None):
    dl = Downloader(f'{path}')
    for ticker in tickers:
        dl.get(filing_type, ticker, after=after_date,before=before_date)

In [8]:
# Uncomment to download
# create_SEC('./Data/Raw_Data/',tickers,"10-Q",after_date='2012-06-01')

In [9]:
def txt_finder():
    txts = []
    for root, dirs, files in os.walk(".", topdown=False):
        for name in files:
            os.path.join(root, name)
            if name[-4:]=='.txt':
                txts.append(f'{root}/{name}')
        for name in dirs:
            os.path.join(root, name)
    return txts
sec_paths = txt_finder()

In [10]:
def path_part(path,before,after):
    beforeNum = path.find(before)+len(before)
    afterNum = path.find(after)
    return path[beforeNum:afterNum]

def sec_df_from_paths(sec_paths,filing_type):
    data = {
        'Ticker':[],
        'DocName':[],
        'DocPath':[],
        'FilingType':[],
        'DocText':[]
    }
    for sec in sec_paths:
        data['Ticker'].append(path_part(sec,'sec-edgar-filings\\',f'\\{filing_type}'))
        data['DocName'].append(path_part(sec,f'\\{filing_type}\\','/full-submission'))
        data['DocPath'].append(sec)
        data['FilingType'].append(filing_type)
        sec_txt = open(sec, "r")
        data['DocText'].append(sec_txt.read())
        sec_txt.close()
    return pd.DataFrame(data)

df_10Q = sec_df_from_paths(sec_paths,'10-Q')

In [11]:
df_10Q.head().tail(2)

,Ticker,DocName,DocPath,FilingType,DocText


In [12]:
import re
def restore_windows_1252_characters(restore_string):
    """
        Replace C1 control characters in the Unicode string s by the
        characters at the corresponding code points in Windows-1252,
        where possible.
    """

    def to_windows_1252(match):
        try:
            return bytes([ord(match.group(0))]).decode('windows-1252')
        except UnicodeDecodeError:
            # No character at the corresponding code point: remove it.
            return ''
        
    return re.sub(r'[\u0080-\u0099]', to_windows_1252, restore_string)

In [13]:
# df_10Q['DocText'].apply(restore_windows_1252_characters)
df_10Q_test = df_10Q.head()
df_10Q_test['DocText'].apply(restore_windows_1252_characters)

Series([], Name: DocText, dtype: float64)

In [14]:
# df_10Q['DocText'].loc[0][:1000]

In [15]:
from sec_api import ExtractorApi
sec_api = os.getenv("sec_api_key")
extractorApi = ExtractorApi(sec_api)

# Tesla 10-K filing
filing_url = "https://www.sec.gov/Archives/edgar/data/1318605/000156459021004599/tsla-10k_20201231.htm"

# get the standardized and cleaned text of section 1A "Risk Factors"
# section_text = extractorApi.get_section(filing_url, "1A", "text")

# print(section_text)

In [16]:
len("CONFORMED PERIOD OF REPORT:\t")

28

In [17]:
def DocText_extraction(text,before,after):
    return text[
        text.find(before)+len(before):
        text.find(after)
        ]

In [18]:
def SEC_date(DocText):
    """Returns Conformed Period of Report when given the text of an SEC report"""
    return DocText_extraction(DocText,"CONFORMED PERIOD OF REPORT:\t","\nFILED AS OF DATE:")
def SEC_CIK(DocText):
    """Returns CENTRAL INDEX KEY when given the text of an SEC report"""
    return DocText_extraction(DocText,"CENTRAL INDEX KEY:\t\t\t","\n\t\tSTANDARD INDUSTRIAL CLASSIFICATION:")
    

In [19]:
def extract_all_SEC_dates(df:pd.DataFrame):
    df['SECdates'] = df['DocText'].apply(SEC_date)
def extract_all_SEC_CIK(df:pd.DataFrame):
    df['SEC_CIK'] = df['DocText'].apply(SEC_CIK)
extract_all_SEC_dates(df_10Q)
extract_all_SEC_CIK(df_10Q)
df_10Q.head()

,Ticker,DocName,DocPath,FilingType,DocText,SECdates,SEC_CIK


In [20]:
def url_maker(df:pd.DataFrame):
    #Example:
    # "https://www.sec.gov/Archives/edgar/data/1318605/000156459021004599/tsla-10k_20201231.htm"
    # url_base:                 https://www.sec.gov/Archives/edgar/data/
    # SEC CIK:                  1318605
    # SEC DOCName:              000156459021004599
    # Ticker:                   tsla
    # Doc Type:                 10k
    # Date (date 2020-02-31):   20201231 


    url_base = "https://www.sec.gov/Archives/edgar/data/"
    SEC_cik = df['SEC_CIK']
    SEC_DocName = df['DocName'].apply(lambda x: re.sub("-","",x))
    ticker = df['Ticker'].apply(str.lower)
    doc_type = df['FilingType'].apply(lambda x: re.sub("-","",x))
    date = df['SECdates']
    return url_base + SEC_cik + "/" + SEC_DocName + "/" + ticker + "-" + doc_type + "_" + date + '.htm'

# urls = url_maker(df_10Q)
# df_10Q['urls'] = urls
# for x in urls:
#     print(x)

In [21]:
df_10Q.head()

,Ticker,DocName,DocPath,FilingType,DocText,SECdates,SEC_CIK


All 10-K and 10-Q sections can be extracted:

    1 - Business
    1A - Risk Factors
    1B - Unresolved Staff Comments
    2 - Properties
    3 - Legal Proceedings
    4 - Mine Safety Disclosures
    5 - Market for Registrant’s Common Equity, Related Stockholder Matters and Issuer Purchases of Equity Securities
    6 - Selected Financial Data (prior to February 2021)
    7 - Management’s Discussion and Analysis of Financial Condition and Results of Operations
    7A - Quantitative and Qualitative Disclosures about Market Risk
    8 - Financial Statements and Supplementary Data
    9 - Changes in and Disagreements with Accountants on Accounting and Financial Disclosure
    9A - Controls and Procedures
    9B - Other Information
    10 - Directors, Executive Officers and Corporate Governance
    11 - Executive Compensation
    12 - Security Ownership of Certain Beneficial Owners and Management and Related Stockholder Matters
    13 - Certain Relationships and Related Transactions, and Director Independence
    14 - Principal Accountant Fees and Services
> https://pypi.org/project/sec-api/


## Other stuff

In [22]:
import libs.sec as sec
import libs.df_helper as dfh
df_sec_model = sec.sec_model_df()[['ticker','periodOfReport','All Text']]
df_sec_model.head(1)

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\jhg55\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


,ticker,periodOfReport,All Text
0,NFLX,2022-03-31,undefined Item 2. Management&#8217;s Discussio...


In [23]:
all_sentiments = dfh.daily_sentiment(df_sec_model,'periodOfReport','All Text')


c:\Users\jhg55\Desktop\Penn\P2G2-ProjectGit\Notebooks\libs\df_helper.py:26: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  vader_df = vader_df.groupby(['ticker',date_col])['ticker','pos','neg','neu','compound'].mean().reset_index()


In [24]:
all_sentiments

,periodOfReport,ticker,pos,neg,neu,compound
0,2020-12-31,ABNB,0.122,0.065,0.812,1.0000
1,2021-03-31,ABNB,0.107,0.028,0.865,0.9999
2,2021-06-30,ABNB,0.082,0.027,0.890,0.9999
3,2021-09-30,ABNB,0.079,0.026,0.895,0.9999
4,2021-12-31,ABNB,0.126,0.069,0.805,1.0000
...,...,...,...,...,...,...
472,2021-03-31,UBER,0.109,0.039,0.852,1.0000
473,2021-06-30,UBER,0.109,0.038,0.853,1.0000
474,2021-09-30,UBER,0.120,0.033,0.847,1.0000
475,2021-12-31,UBER,0.106,0.062,0.832,1.0000


In [25]:
all_sentiments.to_csv('.\Data\Cleaned_Data\sec_sentiment.csv')

In [26]:

for ticker in tickers:
    sentiment = all_sentiments[all_sentiments['ticker']==ticker]
    close_prices = 
    df = pd.concat([sentiment, close], axis = 1)
    uber.ffill(axis = 0, inplace = True)
    airbnb.dropna(inplace = True)
    airbnb.to_csv('./Data/Cleaned_Data/uber.csv')

SyntaxError: invalid syntax (Temp/ipykernel_13300/2727475119.py, line 3)